In [1]:
import sys

sys.path.append('..')
sys.path.append('../task_02')

from dataset import B, Pipeline, C
from dataset.opensets import MNIST
from dataset.models.tf import VNet
from noised_mnist import NoisedMnist
from research import Research
from training import MultipleTraining
import grid

In [2]:
IMAGE_SIZE = 100

In [3]:
mnist = MNIST(batch_class=NoisedMnist)

ExtractingExtractingExtractingExtracting   C:\Users\kozhevin\AppData\Local\Temp\train-labels-idx1-ubyte.gz 
C:\Users\kozhevin\AppData\Local\Temp\t10k-images-idx3-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\t10k-labels-idx1-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\train-images-idx3-ubyte.gz




In [4]:
placeholders_config = {
                       'images': {'shape': (IMAGE_SIZE, IMAGE_SIZE, 1),
                                  'type': 'float32',
                                  'name': 'reshaped_images'},
                
                       'masks': {'shape': (IMAGE_SIZE, IMAGE_SIZE),
                                 'type': 'int32',
                                 'transform': 'ohe', 
                                 'classes': 2,
                                 'name': 'targets'}
                       }

feed_dict = {'images': B('images'),
             'masks': B('masks')}  

In [5]:
config = {'inputs': placeholders_config,
          'input_block/inputs': 'images',
          'batch_norm': {'momentum': 0.1},
          'optimizer': 'Adam'}

In [6]:
load_template = (Pipeline()
                 .random_location(IMAGE_SIZE)
                 .make_masks()
                 .create_noise('mnist_noise', C('level'), 80, 4, 'uniform')
                 .add_noise())

In [7]:
cfg = {
    'name': 'VNet',
    'model': VNet,
    'model_config': config,
    'level': 1
}
mt = MultipleTraining(mnist, cfg, feed_dict, preproc_template=load_template, metrics=[])

In [8]:
mt.run(10, 10, 10)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]


TypeError: 'type' object is not iterable

In [ ]:
p1 = grid.Pair('body/kernel_size', 'A')
v11 = grid.Pair(3)
v12 = grid.Pair(5)

o1 = grid.Option(p1, 3, 5)

p2 = grid.Pair('body/upsample', 'B')
v21 = grid.Pair(dict(layout='tna', factor=2), 'tna')
v22 = grid.Pair(dict(layout='b', factor=2), 'b')

o2 = grid.Option(p2, v21, v22)

In [ ]:
grid_config = o1 + o2

In [ ]:
models = [(VNet, config, grid_config), (UNet, )]

In [ ]:
#res = Research(models, mnist, feed_dict, preproc_template=load_template, aliases=False, name='vnet')
#res.run(batch_size=8, n_reps=10, n_iters=3)

res = Research.load('vnet')

In [ ]:
res.summary()

In [ ]:
res.plot(2, None, hist=False, figsize=(10, 7))

In [ ]:
res.make_video('output.mp4', figsize=(10, 7))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def sorted_plot(data, ax, label, *args, **kwargs):
    ax.plot(sorted(data), label=label, *args, **kwargs)

In [ ]:
res.make_video('output.mp4', func=sorted_plot, figsize=(10, 7))